In [ ]:
!git clone https://github.com/Gurupatil0003/FakeFace

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Flatten,Dense,Conv2D,Conv2DTranspose,Input,Reshape,BatchNormalization,LeakyReLU

In [ ]:
img_ds=tf.keras.utils.image_dataset_from_directory(
    "/content/FakeFace",
    image_size=(64,64),
    batch_size=64,
    label_mode=None
).map(lambda x:(tf.cast(x,tf.float32)/127.5)-1).prefetch(tf.data.AUTOTUNE)

In [ ]:
def build_generator(latent=100):
  return tf.keras.models.Sequential([
      Input((latent,)),
      Dense(4*4*512,use_bias=False),
      Reshape((4,4,512)),
      Conv2DTranspose(256,4,2,padding="same",use_bias=False),
      BatchNormalization(),
      LeakyReLU(),
      Conv2DTranspose(128,4,2,padding="same",use_bias=False),
      BatchNormalization(),
      LeakyReLU(),
      Conv2DTranspose(64,4,2,padding="same",use_bias=False),
      BatchNormalization(),
      LeakyReLU(),
      Conv2DTranspose(3,4,2,padding="same",activation="tanh")



  ])

def build_discriminator(img=64):
   return tf.keras.models.Sequential([
        Input((img,img,3)),
        Conv2D(64,4,2,padding="same"),LeakyReLU(0.2),
        Conv2D(128,4,2,padding="same"),LeakyReLU(0.2),
        Conv2D(256,4,2,padding="same"),LeakyReLU(0.2),
        Conv2D(512,4,2,padding="same"),LeakyReLU(0.2),
        Flatten(),
        Dense(1)
   ])

G,D=build_generator(),build_discriminator()


In [ ]:
loss_fn=tf.keras.losses.BinaryCrossentropy(from_logits=True)
g_opt=tf.keras.optimizers.Adam(2e-4,0.5)
d_opt=tf.keras.optimizers.Adam(2e-4,0.5)



@tf.function
def data_test(real):
  noise=tf.random.normal([64,100])
  with tf.GradientTape() as g_tape:
    fake=G(noise,training=True)
    fake_logits=D(fake,training=True)
    g_loss=loss_fn(tf.ones_like(fake_logits),fake_logits)

  with tf.GradientTape() as d_tape:
    fake=G(noise,training=True)
    real_logits=D(real,training=True)
    fake_logits=D(fake,training=True)

    real_loss=loss_fn(tf.ones_like(real_logits),real_logits)
    fake_loss=loss_fn(tf.zeros_like(fake_logits),fake_logits)
    d_loss = real_loss + fake_loss


  g_opt.apply_gradients(zip(g_tape.gradient(g_loss,G.trainable_variables),G.trainable_variables))
  d_opt.apply_gradients(zip(d_tape.gradient(d_loss,D.trainable_variables),D.trainable_variables))
  return g_loss, d_loss

In [ ]:
EPOCHS=70
import matplotlib.pyplot as plt
seed=tf.random.normal([64,100])
for epoch in range(EPOCHS):
  for real in img_ds:
    g_loss, d_loss=data_test(real)
    print(f"{epoch+1}/{EPOCHS} G:{g_loss} , D:{d_loss}")
    if(epoch+1)%2==0:
      img=(G(seed,training=False))
      plt.figure(figsize=(4,4))
      for i in range(16):
        plt.subplot(4,4,i+1)
        plt.imshow((img[i]+1)/2)
        plt.axis("off")
      plt.show()